# Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

# 1. Импортируем библиотеку и данные.

In [258]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [259]:
df = pd.read_csv('desktop/data.csv')

In [260]:
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## 2. Предобработка данных

Определить и заполнить пропущенные значения

In [261]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения имются в таблицах days_employed и total_income обе таблицы являются количественными признаками, значит пропущенные значения можно заполнить средним значениям или медианой.

Но перед этим посмотрим на возможные причины появления пропусков в данных.

In [262]:
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Видим, что пропуски days_employed и total_income неслучайны, так как они зависят друг от друга.

In [263]:
df.days_employed = df.days_employed.fillna(df.days_employed.median())
df.total_income = df.total_income.fillna(df.total_income.median())

Заменить вещественные типы данных на целочисленные.

In [265]:
df.days_employed = df.days_employed.astype(int).abs()
df.total_income = df.total_income.astype(int)
df.children = df.children.abs()

Удалите дубликаты

Дуплекаты появляются в данных из-за повторных представлений, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

Посмотрим сколько дупликатов у нас есть в данных с помощью duplicate().sum().

In [266]:
df.duplicated().sum()

54

Видим, что дуплекатов 54, они нам не нужны и мы их удалим с помощью drop_duplicates()

In [267]:
df = df.drop_duplicates()

Выделить леммы в значениях столбца с целями получения кредита.

In [268]:
m = Mystem()

In [269]:
lemass = m.lemmatize

In [270]:
lemmas = df.purpose

In [271]:
Counter(lemmas.sort_values(ascending = False))

Counter({'сыграть свадьбу': 769,
         'строительство собственной недвижимости': 635,
         'строительство недвижимости': 619,
         'строительство жилой недвижимости': 625,
         'сделка с подержанным автомобилем': 486,
         'сделка с автомобилем': 455,
         'свой автомобиль': 479,
         'свадьба': 793,
         'ремонт жилью': 607,
         'профильное образование': 436,
         'приобретение автомобиля': 461,
         'получение образования': 442,
         'получение дополнительного образования': 447,
         'получение высшего образования': 426,
         'покупка своего жилья': 620,
         'покупка недвижимости': 621,
         'покупка коммерческой недвижимости': 662,
         'покупка жилья для семьи': 638,
         'покупка жилья для сдачи': 652,
         'покупка жилья': 646,
         'покупка жилой недвижимости': 606,
         'операции со своей недвижимостью': 627,
         'операции с недвижимостью': 675,
         'операции с коммерческой недвижимос

Цель сыграть свадьбу является самой популярной.

## 3.Категоризация данных

In [272]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [333]:
def category_purpose(purpose):
    if 'недвижимость' in m.lemmatize(purpose) or 'жилье' in m.lemmatize(purpose):
        return 'недвижимость'
    if 'свадьба' in m.lemmatize(purpose) or 'сыграть' in m.lemmatize(purpose):
        return 'свадьба'
    if 'автомобиль' in m.lemmatize(purpose):
        return 'автомобиль'
    if 'образование'in m.lemmatize(purpose):
        return 'образование'
    if 'ремонт' in m.lemmatize(purpose):
        return 'ремонт'
    else:
        return 'прочее'
df['category_purpose'] = df['purpose'].apply(category_purpose)
df['income_cut'] = pd.cut(df['total_income'],[0,120000,150000,300000,500000,np.inf])

In [334]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,category_purpose,income_cut
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,"(150000.0, 300000.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,"(0.0, 120000.0]"
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,"(120000.0, 150000.0]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,"(150000.0, 300000.0]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,"(150000.0, 300000.0]"


Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [288]:
df.groupby('children')\
      .agg({'debt': 'mean'}).round(3)*100

,debt
children,
0,7.5
1,9.2
2,9.5
3,8.2
4,9.8
5,0.0
20,10.5


На ответ на этот вовпрос я сгруппировал таблицу с количеством детей , а в качестве значений взял колонку debt и посчитал среднее по ней.

У тех у кого 5 детей не имеют задолжностей следовательно являются хорошими заёмщиками. Люди у которых 20 детей являются 'худшими' заемщиками 10.5%. Однако, нельзя сказать , что количество детей влияет на возврат кредита, так как у тех у кого есть 2 детей имеют задолженость и являются не очень хорошими заемщиками.

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [298]:
df.groupby('family_status')\
  .agg({'debt': 'mean'}).round(3)*100

,debt
family_status,
Не женат / не замужем,9.8
в разводе,7.1
вдовец / вдова,6.6
гражданский брак,9.3
женат / замужем,7.5


На ответ на этот вовпрос я сгруппировал таблицу по таблице family_status , а в качестве значений взял колонку debt и посчитал среднее по ней.

'Худшими' заемщиками являются не женатые/не замужем и те, кто состоит в гражданском браке. Можно предположить, что они являются менее ответственные, чем люди, которые состоят или были в браке.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [335]:
df.groupby('income_cut')\
  .agg({'debt': 'mean'}).round(3)*100

,debt
income_cut,
"(0.0, 120000.0]",8.0
"(120000.0, 150000.0]",8.5
"(150000.0, 300000.0]",8.0
"(300000.0, 500000.0]",7.3
"(500000.0, inf]",6.3


Люди, у которых доход максимальный возвращают долг вовремя. И люди с доход более 300000 отдают долг чаще. Значит, между уровнем дохода и возвратом кредита в срок есть зависимость.

Как разные цели кредита влияют на его возврат в срок?

In [336]:
df.groupby('category_purpose')\
  .agg({'debt': 'mean'}).round(3)*100

,debt
category_purpose,
автомобиль,9.4
недвижимость,7.2
образование,9.2
свадьба,8.0


Люди, у которых целью является автомобиль или образование являются 'худшими' заёмщиками. А те, у которых целью является недвижимость являются хорошими.

## Общий вывод

В данной работе я обработал данные от пропусков и дупликатов. Провел лемматизацию и на основе их сделал категоризацию по целям кредита. Также провел категоризацию по доходам.

Ответил на вопросы:

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Нет, в среднем семья в которой 5 детей не имеют задолжности по возврату. Однока в семье в которой 2 детей имеют задолжность и являются плохими. Я не заметил никакой зависимости.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

В среднем, люди , которые находятся в статусе женат/замужем и в разводе являются хорошими.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Да, чем выше доход, тем выше шанс возврата.

- Как разные цели кредита влияют на его возврат в срок?

Люди с целью приобрести недвижимость являются хорошими заемщиками.
